# Contextualização do Problema

Os estabelecimentos financeiros recebem muitas solicitações de cartões de crédito. Muitas delas são negadas devido a fatores variados, como dívidas elevadas, renda baixa ou numerosas consultas no histórico de crédito de uma pessoa, entre outros. Analisar essas solicitações manualmente é uma tarefa chata, sujeita a falhas e demorada (e tempo é dinheiro!). Felizmente, essa atividade pode ser automatizada com o uso da inteligência artificial e praticamente todos os bancos comerciais o fazem hoje em dia. Neste Notebook, meu objetivo é construir um sistema automático para aprovação de cartão de crédito utilizando técnicas de inteligência artificial, assim como os bancos reais fazem.

In [25]:
import pandas as pd

# Load dataset
dados = pd.read_csv("cc_approvals.data", header=None)

# Inspect data
display(dados)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,b,21.08,10.085,y,p,e,h,1.25,f,f,0,f,g,00260,0,-
686,a,22.67,0.750,u,g,c,v,2.00,f,t,2,t,g,00200,394,-
687,a,25.25,13.500,y,p,ff,ff,2.00,f,t,1,t,g,00200,1,-
688,b,17.92,0.205,u,g,aa,v,0.04,f,f,0,f,g,00280,750,-


# Inspecionando os Dados

A apresentação inicial pode parecer complicada, mas vamos tentar identificar as funcionalidades mais importantes de um aplicativo de cartão de crédito. As características deste conjunto de dados foram anonimizadas para proteger a privacidade, mas podemos ter uma uma ideia geral das características do dataset através deste link:
http://rstudio-pubs-static.s3.amazonaws.com/73039_9946de135c0a49daa7a0a9eda4a67a72.html.
Os recursos prováveis em um aplicativo de cartão de crédito típico são Gênero, Idade, Dívida, Estado Civil, Banco, Cliente, Grau de Educação, Etnia, Anos de Emprego, Prioridade de Inadimplência, Empregado, Pontuação de Crédito, Licença de Condução, Cidadania, CEP, Renda e, finalmente, o Status de Aprovação. Isso nos dá um bom ponto de partida e podemos mapear esses recursos em relação às colunas na saída.

Conforme podemos observar em nossa primeira análise dos dados, o conjunto de dados possui uma combinação de características numéricas e não numéricas. Isso pode ser corrigido com algum pré-processamento, mas antes de fazermos isso, vamos aprender um pouco mais sobre o conjunto de dados para ver se há outros problemas de conjunto de dados que precisam ser corrigidos.

In [21]:
# Print summary statistics
descricao_dados = dados.describe()
print(descricao_dados)

print('\n')

# Print DataFrame information
dados_info = dados.info()
print(dados_info)

               2           7          10             14
count  690.000000  690.000000  690.00000     690.000000
mean     4.758725    2.223406    2.40000    1017.385507
std      4.978163    3.346513    4.86294    5210.102598
min      0.000000    0.000000    0.00000       0.000000
25%      1.000000    0.165000    0.00000       0.000000
50%      2.750000    1.000000    0.00000       5.000000
75%      7.207500    2.625000    3.00000     395.500000
max     28.000000   28.500000   67.00000  100000.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       690 non-null    object 
 1   1       690 non-null    object 
 2   2       690 non-null    float64
 3   3       690 non-null    object 
 4   4       690 non-null    object 
 5   5       690 non-null    object 
 6   6       690 non-null    object 
 7   7       690 non-null    float64
 8   8       690 no

# Dividindo o Dataset em Dados de Treino e Teste

Agora, dividirei o dataset em conjuntos de treino e  teste para preparar os dados para duas fases diferentes da modelagem de aprendizado de máquina: treinamento e teste. Idealmente, nenhuma informação dos dados de teste deve ser usada para pré-processar os dados de treinamento ou deve ser usada para direcionar o processo de treinamento de um modelo de aprendizado de máquina. Portanto, primeiro dividimos os dados e depois os pré-processamos.

Além disso, recursos como DriversLicense e ZipCode (ver link disponibilizado na sessão anterior para melhor contextualização) não são tão relevantes quanto os outros recursos no conjunto de dados para prever aprovações de cartão de crédito, desta forma, podemos eliminá-los para projetar nosso modelo de aprendizado de máquina com o melhor precisão. Na literatura de Ciência de Dados, isso costuma ser chamado de seleção de recursos.

In [26]:
# Importando train_test_split
from sklearn.model_selection import train_test_split

# Dropando as variáveis 11 e 13
dados = dados.drop([11, 13], axis=1)

# Split into train and test sets
dados_train, dados_test = train_test_split(dados, test_size=0.33, random_state=42)

In [27]:
dados_train

,0,1,2,3,4,5,6,7,8,9,10,12,14,15
382,a,24.33,2.500,y,p,i,bb,4.500,f,f,0,g,456,-
137,b,33.58,2.750,u,g,m,v,4.250,t,t,6,g,0,+
346,?,32.25,1.500,u,g,c,v,0.250,f,f,0,g,122,-
326,b,30.17,1.085,y,p,c,v,0.040,f,f,0,g,179,-
33,a,36.75,5.125,u,g,e,v,5.000,t,f,0,g,4000,+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,b,34.83,4.000,u,g,d,bb,12.500,t,f,0,g,0,-
106,b,28.75,1.165,u,g,k,v,0.500,t,f,0,s,0,-
270,b,37.58,0.000,?,?,?,?,0.000,f,f,0,p,0,+
435,b,19.00,0.000,y,p,ff,ff,0.000,f,t,4,g,1,-


In [29]:
dados_test

,0,1,2,3,4,5,6,7,8,9,10,12,14,15
286,a,?,1.50,u,g,ff,ff,0.000,f,t,2,g,105,-
511,a,46.00,4.00,u,g,j,j,0.000,t,f,0,g,960,+
257,b,20.00,0.00,u,g,d,v,0.500,f,f,0,g,0,-
336,b,47.33,6.50,u,g,c,v,1.000,f,f,0,g,228,-
318,b,19.17,0.00,y,p,m,bb,0.000,f,f,0,s,1,+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,a,20.83,0.50,y,p,e,dd,1.000,f,f,0,g,0,-
234,a,58.42,21.00,u,g,i,bb,10.000,t,t,13,g,6700,+
644,b,36.17,0.42,y,p,w,v,0.290,f,f,0,g,2,-
271,b,32.33,2.50,u,g,c,v,1.250,f,f,0,g,0,-


# Manipulando Valores Ausentes

Agora que os dados já estão divididos em treino e teste, podemos solucionar alguns problemas previamente identificados no dataset, tais como:

- O Dataset possui dados numéricos e não numéricos (dos dipos int64, float64 e Object).
- Mais específicamente, as colunas 2, 7, 10 e 14 possuem dados do tipo int64, int64, float64 e float64 respectivamente, enquanto todas as demais variáveis são não-numéricas.
- É possível observar uma grande dispersão nos cados de algumas variáveis, sendo que algumas tem vão de intervalos de 0 a 28, outras de 2 a 67, ou ainda mais dispersos, de 1017 a 100000, sabendo disto, também podemos obter informações estatísticas importantes a partir das variáveis numéricas, como: valor máximo, mínimo, média, mediana, entre outros.
- Por fim, é possível observar a existência de valores susentes, que são identificador por uma interrogação "?", nesta estpa do projeto, vou substituir temporáriamente está interrogação por "NaN".

In [23]:
# Import numpy
import numpy as np

# Substituindo "?" por "NaN" nos dados de treino e teste
dados = dados_train.replace('?', np.NaN)
dados_test = dados_test.replace('?', np.NaN)

In [24]:
dados_test

,0,1,2,3,4,5,6,7,8,9,10,12,14,15
286,a,NaN,1.50,u,g,ff,ff,0.000,f,t,2,g,105,-
511,a,46.00,4.00,u,g,j,j,0.000,t,f,0,g,960,+
257,b,20.00,0.00,u,g,d,v,0.500,f,f,0,g,0,-
336,b,47.33,6.50,u,g,c,v,1.000,f,f,0,g,228,-
318,b,19.17,0.00,y,p,m,bb,0.000,f,f,0,s,1,+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,a,20.83,0.50,y,p,e,dd,1.000,f,f,0,g,0,-
234,a,58.42,21.00,u,g,i,bb,10.000,t,t,13,g,6700,+
644,b,36.17,0.42,y,p,w,v,0.290,f,f,0,g,2,-
271,b,32.33,2.50,u,g,c,v,1.250,f,f,0,g,0,-
